# EVAC Assessment 2 - Society-based Cooperation

In [ ]:
import random
from enums import Society
from deap import creator, base, tools
import itertools
import logging

In [ ]:
OUTCOMES = {
    # Saints cooperate with everyone
    (Society.SAINTS, Society.SAINTS):           (4,4), # Both cooperate
    (Society.SAINTS, Society.BUDDIES):          (0,6), # Buddies are selfish
    (Society.SAINTS, Society.FIGHT_CLUB):       (4,4), # Both cooperate
    (Society.SAINTS, Society.VANDALS):          (0,6), # Vandals are selfish

    # Buddies only cooperate with each other
    (Society.BUDDIES, Society.SAINTS):          (6,0), # Buddies are selfish
    (Society.BUDDIES, Society.BUDDIES):         (4,4), # Both cooperate
    (Society.BUDDIES, Society.FIGHT_CLUB):      (6,0), # Buddies are selfish
    (Society.BUDDIES, Society.VANDALS):         (1,1), # Both selfish
    
    # Fight club cooperate with everyone but themselves
    (Society.FIGHT_CLUB, Society.SAINTS):       (4,4), # Both cooperate
    (Society.FIGHT_CLUB, Society.BUDDIES):      (0,6), # Buddies are selfish
    (Society.FIGHT_CLUB, Society.FIGHT_CLUB):   (1,1), # Both selfish
    (Society.FIGHT_CLUB, Society.VANDALS):      (0,6), # Vandals are selfish
    
    # Vandals cooperate with no one
    (Society.VANDALS, Society.SAINTS):          (6,0), # Vandals are selfish
    (Society.VANDALS, Society.BUDDIES):         (1,1), # Both selfish
    (Society.VANDALS, Society.FIGHT_CLUB):      (6,0), # Vandals are selfish
    (Society.VANDALS, Society.VANDALS):         (1,1), # Both selfish
}

In [ ]:
IND_SIZE = 4**6 # will actually be 4^6
POP_SIZE = 5
NUM_ROUNDS = 1

def reset_individuals(population):
    for indiv in population:
        indiv.total_score = 0
        indiv.rounds_played = 0
        indiv.society = random.choice(list(Society))
        indiv.history = [random.choice([society.value for society in Society]) for i in range(6)]


def play_round(indiv1, indiv2):
    indiv1.rounds_played += 1
    indiv2.rounds_played += 1
    
    payoffs = ROUND_OUTCOMES[(indiv1.society, indiv2.society)]
    indiv1.total_score += payoffs[0]
    indiv2.total_score += payoffs[1]

    new_match = [indiv1.society.value, indiv2.society.value]
    indiv1.history = indiv1.history[2:6] + new_match
    indiv2.history = indiv2.history[2:6] + new_match[::-1]

    chr1_index = HISTORY_LOOKUP[tuple(indiv1.history)]
    indiv1.society = Society(indiv1[chr1_index])

    chr2_index = HISTORY_LOOKUP[tuple(indiv2.history)]
    indiv2.society = Society(indiv1[chr2_index])

def evaluate_agents(population):
    logging.info("Running the game")
    for i in range(NUM_ROUNDS):
        indiv1 = random.choice(population)
        indiv2 = random.choice(population)
        while indiv2 == indiv1:
            indiv2 = random.choice(population)

        print_indiv(indiv1)
        print_indiv(indiv2)
        print("------------")
        play_round(indiv1, indiv2)
        print_indiv(indiv1)
        print_indiv(indiv2)

def print_indiv(indiv):
    print("~~~~~~~~~~~~~~~~~~~~~~~")
    print(f"{indiv.total_score=}")
    print(f"{indiv.rounds_played=}")
    print(f"{indiv.society=}")
    print(f"{indiv.history=}")
    print("~~~~~~~~~~~~~~~~~~~~~~~")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax, 
    total_score=0, rounds_played=0, society=Society.SAINTS, 
    history = [0 for i in range(6)])

toolbox = base.Toolbox()
toolbox.register("int_attribute", random.choice, [society.value for society in Society])

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.int_attribute, n = IND_SIZE)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_agents)

toolbox.register("selectTournament", tools.selTournament, tournsize = 2)

toolbox.register("mateTwoPoints", tools.cxTwoPoint)
toolbox.register("mutateShuffleIndexes", tools.mutShuffleIndexes, indpb = 0.05)

population = toolbox.population(POP_SIZE)
reset_individuals(population)
toolbox.evaluate(population)
best = tools.selBest(population, 1)